# <span style="color:#3399ff">Beautiful Soup - Exemplos</span>

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime 
import numpy as np

In [ ]:
html_doc = """<html><head><title>The Dormouse's story</title></head>
<body>
<p class="title"><b>The Dormouse's story</b></p>

<p class="story">Once upon a time there were three little sisters; and their names were
<a href="http://example.com/elsie" class="sister" id="link1">Elsie</a>,
<a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> and
<a href="http://example.com/tillie" class="sister" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>

<p class="story">...</p>
"""

In [ ]:
soup = BeautifulSoup(html_doc, 'html.parser')

In [ ]:
soup

In [ ]:
print(soup.title)

In [ ]:
print(soup.head)

In [ ]:
print(soup.body.p)

In [ ]:
soup.find_all('p')

In [ ]:
soup.find_all('p', 'title')

In [ ]:
soup.find_all('p', class_='story')

In [ ]:
soup.find_all('a', class_='sister')

In [ ]:
soup.find_all('a', id='link1')

In [ ]:
soup.find_all('a', id='link1')[0].string

In [ ]:
soup.find_all('a', id='link1')[0].get_text()

# <span style="color:#3399ff">Beautiful Soup - Prática I</span>

In [6]:
# id
# product_name
# product_type
# product_composition
# preco

In [7]:
url = 'https://www2.hm.com/en_us/men/products/jeans.html'

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
page = requests.get(url, headers=headers)

In [8]:
soup = BeautifulSoup(page.text, 'html.parser')

In [9]:
products = soup.find('ul', class_='products-listing small')

In [10]:
product_list = products.find_all('article', class_='hm-product-item')

In [11]:
len(product_list)

36

In [12]:
# product_id
product_id = [p.get('data-articlecode') for p in product_list]

# product_category
product_category = [p.get('data-category') for p in product_list]

In [13]:
# product_name
product_list = products.find_all('a', class_='link')
product_name = [p.get_text() for p in product_list] 

In [14]:
# product_price
product_list = products.find_all('span', class_='price regular')
product_price = [p.get_text() for p in product_list]

In [15]:
data = pd.DataFrame([product_id , product_category, product_name, product_price]).T
data.columns = ['product_id' , 'product_category', 'product_name', 'product_price']
data.head()

,product_id,product_category,product_name,product_price
0,1024256001,men_jeans_slim,Slim Jeans,$ 19.99
1,0985159001,men_jeans_skinny,Skinny Jeans,$ 24.99
2,1024256003,men_jeans_slim,Slim Jeans,$ 19.99
3,0690449043,men_jeans_ripped,Skinny Jeans,$ 39.99
4,0690449036,men_jeans_ripped,Skinny Jeans,$ 39.99


In [ ]:
# scrapy datetime
data['scrapy_datetime'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
data.head()

# <span style="color:#3399ff">Beautiful Soup - Prática II</span>

- Sempre olhar a paginação na vitrine de produtos.

In [ ]:
url = 'https://www2.hm.com/en_us/men/products/jeans.html'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
page = requests.get(url, headers=headers)

In [ ]:
soup = BeautifulSoup(page.text, 'html.parser')

In [ ]:
total_item = soup.find_all('h2', class_='load-more-heading')[0].get('data-total')

In [ ]:
total_item

In [ ]:
int(total_item)

- De quantas páginas preciso para capturar todos os itens?

In [ ]:
page_number = np.round(int(total_item)/36)
page_number

In [ ]:
url02 = url + '?page-size=' + str(int(page_number*36))
url02

# <span style="color:#3399ff">Beautiful Soup - Prática III</span>

- Aqui, vamos coletar as informações para cada produto.

- Fazendo para um produto para testar:

In [ ]:
url = 'https://www2.hm.com/en_us/productpage.1024256001.html'
    
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
page = requests.get(url, headers=headers)

In [ ]:
soup = BeautifulSoup(page.text, 'html.parser')

In [ ]:
type(soup)

In [ ]:
# color name - primeira posição
soup.find_all('a', class_='filter-option miniature')[0].get('data-color')

In [ ]:
# color name
product_color_list = soup.find_all('a', class_='filter-option miniature')
color_name = [p.get('data-color') for p in product_color_list]

# product_id
product_id = [p.get('data-articlecode') for p in product_color_list]

df_color = pd.DataFrame([product_id, color_name]).T
df_color.columns = ['product_id', 'color_name']
df_color

In [ ]:
product_color_list[0]

In [ ]:
color_name

In [ ]:
product_id

In [ ]:
# composition
soup.find_all('div', class_='pdp-description-list-item')

In [ ]:
type(soup.find_all('div', class_='pdp-description-list-item')[1])

In [ ]:
soup.find_all('div', class_='pdp-description-list-item')[1].get_text().split('\n')

- **Retirando essas posições vazias:**

In [ ]:
list(filter(None, soup.find_all('div', class_='pdp-description-list-item')[1].get_text().split('\n')))

- **Construindo linha e coluna:**

In [3]:
product_composition_list = soup.find_all('div', class_='pdp-description-list-item')
product_composition = [list(filter(None, p.get_text().split('\n'))) for p in product_composition_list]
df_aux = pd.DataFrame(product_composition).T
df_aux

,0,1,2,3
0,Size,Fit,Composition,Art. No.
1,"The model is 185cm/6'1"" and wears a size 31/32",Slim fit,"Pocket lining: Polyester 65%, Cotton 35%",1024256001
2,None,None,"Shell: Cotton 99%, Spandex 1%",None


In [ ]:
df_aux.iloc[0]

In [ ]:
# rename data frame
df_aux.columns = df_aux.iloc[0]
df_aux

In [ ]:
# delete first row
df_aux = df_aux.iloc[1:].copy().reset_index()
df_aux

In [ ]:
df_aux = df_aux[['Fit', 'Composition', 'Art. No.']]
df_aux

- **Substituindo o None:**

In [ ]:
df_aux

In [ ]:
df_aux = df_aux.fillna(method='ffill')
df_aux

In [ ]:
df_composition = df_aux
df_composition

In [ ]:
# generate style id + color id
df_composition['style_id'] = df_composition['Art. No.'].apply(lambda x: x[:-3])
df_composition['color_id'] = df_composition['Art. No.'].apply(lambda x: x[-3:])
df_composition

In [ ]:
# generate style id + color id
df_color['style_id'] = df_color['product_id'].apply(lambda x: x[:-3])
df_color['color_id'] = df_color['product_id'].apply(lambda x: x[-3:])
df_color

In [ ]:
# Join
pd.merge(df_color, df_composition[['style_id', 'Fit', 'Composition']], how='left', on='style_id')

# <span style="color:#3399ff">Juntando todos os passos anteriores </span>

## <span style="color:#ff8000">0. Packages </span>

In [1]:
import requests
import pandas as pd
from datetime import datetime
from bs4 import BeautifulSoup

## <span style="color:#ff8000">1. One Product </span>

In [3]:
# API Requests
url = 'https://www2.hm.com/en_us/productpage.1024256001.html'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5)AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
page = requests.get( url, headers=headers )


# Beautiful Soup object
soup = BeautifulSoup(page.text, 'html.parser')

# ================== color name =====================================
product_list = soup.find_all('a', class_='filter-option miniature')
color_name = [p.get('data-color') for p in product_list]

# color name
product_color_list = soup.find_all('a', class_='filter-option miniature')
color_name = [p.get('data-color') for p in product_color_list]

# product_id
product_id = [p.get('data-articlecode') for p in product_color_list]

df_color = pd.DataFrame([product_id, color_name]).T
df_color.columns = ['product_id', 'color_name']

# generate style id + color id
df_color['style_id'] = df_color['product_id'].apply(lambda x: x[:-3])
df_color['color_id'] = df_color['product_id'].apply(lambda x: x[-3:])

# ================== composition =====================================
product_composition_list = soup.find_all('div', class_='pdp-description-list-item')
product_composition = [list(filter(None, p.get_text().split('\n'))) for p in product_composition_list]


# rename dataframe
df_composition = pd.DataFrame(product_composition).T
df_composition.columns = df_composition.iloc[0]


# delete first row
df_composition = df_composition.iloc[1:].fillna(method='ffill')
df_composition = df_composition[['Fit', 'Composition', 'Art. No.']]

# generate style id + color id
df_composition['style_id'] = df_composition['Art. No.'].apply(lambda x: x[:-3])
df_composition['color_id'] = df_composition['Art. No.'].apply(lambda x: x[-3:])

# Merge data color + composition
data_sku = pd.merge(df_color, df_composition[['style_id', 'Fit', 'Composition']], how='left', on='style_id')


In [4]:
data_sku

,product_id,color_name,style_id,color_id,Fit,Composition
0,1024256002,Light denim blue,1024256,002,Slim fit,"Pocket lining: Polyester 65%, Cotton 35%"
1,1024256002,Light denim blue,1024256,002,Slim fit,"Shell: Cotton 99%, Spandex 1%"
2,1024256003,Light denim blue,1024256,003,Slim fit,"Pocket lining: Polyester 65%, Cotton 35%"
3,1024256003,Light denim blue,1024256,003,Slim fit,"Shell: Cotton 99%, Spandex 1%"
4,1024256004,Denim blue,1024256,004,Slim fit,"Pocket lining: Polyester 65%, Cotton 35%"
5,1024256004,Denim blue,1024256,004,Slim fit,"Shell: Cotton 99%, Spandex 1%"
6,1024256005,Dark blue,1024256,005,Slim fit,"Pocket lining: Polyester 65%, Cotton 35%"
7,1024256005,Dark blue,1024256,005,Slim fit,"Shell: Cotton 99%, Spandex 1%"
8,1024256006,Dark denim blue,1024256,006,Slim fit,"Pocket lining: Polyester 65%, Cotton 35%"
9,1024256006,Dark denim blue,1024256,006,Slim fit,"Shell: Cotton 99%, Spandex 1%"


## <span style="color:#ff8000">2. Multiple Products </span>

In [16]:
data.head()

,product_id,product_category,product_name,product_price
0,1024256001,men_jeans_slim,Slim Jeans,$ 19.99
1,0985159001,men_jeans_skinny,Skinny Jeans,$ 24.99
2,1024256003,men_jeans_slim,Slim Jeans,$ 19.99
3,0690449043,men_jeans_ripped,Skinny Jeans,$ 39.99
4,0690449036,men_jeans_ripped,Skinny Jeans,$ 39.99


In [20]:
# ===================================== TESTES ====================================
for i in range(len(data)):

    # API Requests
    url = 'https://www2.hm.com/en_us/productpage.' + data.loc[i, 'product_id'] + '.html'
    print(url)

https://www2.hm.com/en_us/productpage.1024256001.html
https://www2.hm.com/en_us/productpage.0985159001.html
https://www2.hm.com/en_us/productpage.1024256003.html
https://www2.hm.com/en_us/productpage.0690449043.html
https://www2.hm.com/en_us/productpage.0690449036.html
https://www2.hm.com/en_us/productpage.0690449056.html
https://www2.hm.com/en_us/productpage.1024256004.html
https://www2.hm.com/en_us/productpage.0690449022.html
https://www2.hm.com/en_us/productpage.0690449051.html
https://www2.hm.com/en_us/productpage.1004199002.html
https://www2.hm.com/en_us/productpage.1004199005.html
https://www2.hm.com/en_us/productpage.0985159004.html
https://www2.hm.com/en_us/productpage.1024256007.html
https://www2.hm.com/en_us/productpage.1008110001.html
https://www2.hm.com/en_us/productpage.0985159002.html
https://www2.hm.com/en_us/productpage.0985159005.html
https://www2.hm.com/en_us/productpage.0875105023.html
https://www2.hm.com/en_us/productpage.1004199001.html
https://www2.hm.com/en_us/pr

In [30]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5)AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

# empty data frame to store all the needed information
df_details = pd.DataFrame()


for i in range(len(data)):

    # API Requests
    url = 'https://www2.hm.com/en_us/productpage.' + data.loc[i, 'product_id'] + '.html'
    page = requests.get( url, headers=headers )


    # Beautiful Soup object
    soup = BeautifulSoup(page.text, 'html.parser')

    # ================== color name =====================================
    product_list = soup.find_all('a', class_='filter-option miniature')
    color_name = [p.get('data-color') for p in product_list]

    # color name
    product_color_list = soup.find_all('a', class_='filter-option miniature')
    color_name = [p.get('data-color') for p in product_color_list]

    # product_id
    product_id = [p.get('data-articlecode') for p in product_color_list]

    df_color = pd.DataFrame([product_id, color_name]).T
    df_color.columns = ['product_id', 'color_name']

    # generate style id + color id
    df_color['style_id'] = df_color['product_id'].apply(lambda x: x[:-3])
    df_color['color_id'] = df_color['product_id'].apply(lambda x: x[-3:])

    # ================== composition =====================================
    product_composition_list = soup.find_all('div', class_='pdp-description-list-item')
    product_composition = [list(filter(None, p.get_text().split('\n'))) for p in product_composition_list]


    # rename dataframe
    df_composition = pd.DataFrame(product_composition).T
    df_composition.columns = df_composition.iloc[0]


    # delete first row
    df_composition = df_composition.iloc[1:].fillna(method='ffill')
    df_composition = df_composition[['Fit', 'Composition', 'Art. No.']]

    # generate style id + color id
    df_composition['style_id'] = df_composition['Art. No.'].apply(lambda x: x[:-3])
    df_composition['color_id'] = df_composition['Art. No.'].apply(lambda x: x[-3:])

    # Merge data color + composition
    data_sku = pd.merge(df_color, df_composition[['style_id', 'Fit', 'Composition']], how='left', on='style_id')
    
    # Products' details
    df_details = pd.concat([df_details, data_sku], axis=0)
    
# Join showroom data + details
data['style_id'] = data['product_id'].apply(lambda x: x[:-3])
data['color_id'] = data['product_id'].apply(lambda x: x[-3:])

data_raw = pd.merge(data, df_details[['style_id', 'color_name', 'Fit', 'Composition']], how='left', on='style_id')


In [29]:
data_raw.head()

,product_id,product_category,product_name,product_price,style_id,color_id,color_name,Fit,Composition
0,1024256001,men_jeans_slim,Slim Jeans,$ 19.99,1024256,001,Light denim blue,Slim fit,"Pocket lining: Polyester 65%, Cotton 35%"
1,1024256001,men_jeans_slim,Slim Jeans,$ 19.99,1024256,001,Light denim blue,Slim fit,"Shell: Cotton 99%, Spandex 1%"
2,1024256001,men_jeans_slim,Slim Jeans,$ 19.99,1024256,001,Light denim blue,Slim fit,"Pocket lining: Polyester 65%, Cotton 35%"
3,1024256001,men_jeans_slim,Slim Jeans,$ 19.99,1024256,001,Light denim blue,Slim fit,"Shell: Cotton 99%, Spandex 1%"
4,1024256001,men_jeans_slim,Slim Jeans,$ 19.99,1024256,001,Denim blue,Slim fit,"Pocket lining: Polyester 65%, Cotton 35%"


# <span style="color:#0066ff">Exercícios Práticos<span>

**1. Coletar os seguintes dados da página: https://books.toscrape.com**
- Catálogo:
    + Classics
    + Science Fiction
    + Humor
    + Business

- Coletar os seguintes dados de cada livro:
    + Nome do livro
    + Preço em libras
    + Avaliação dos consumidores
    + Disponível em estoque

**2. Entregável:**

- Faça um plano escrito para cada uma das perguntas de negócio, contendo:
    - Saída: A simulação da tabela e gráfico final.
    - Processo: A sequência de passos organizada pela lógica de execução
    - Entrada: O link para as fontes de dados.
    

- Uma csv com todas as informação de todos os catálogos.